# Telco Churn

### image

In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
import itertools

# visualization
import seaborn as sns
import matplotlib.pyplot as plt


# classification models
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

# custom modules
import modeling as md
import prepare
import acquire
import exploration
import split

# global variable, will never change
alpha = 0.05

# Project Planning (readme)

* Create acquire.py module to obtain data from the codeup database, cache them for later use.
* Create prepare.py module to prepare/clean the data (handling missing values, encoding for the model, etc.)
* Create/use functions to explore variables through visualization and statistical testing, write down any insights gained through both methods. Hypothesis required for each statistical test.
* Establish baseline accuracy for our target variable.
* Create several models using training data, then evaluate the models on both training and validate data.
* Take the best performing model and test it on the test data.
* At the same time, save the predictions to a csv with the values of customer_id, the probability of our target variable, and the model's prediction.
* Document conclusions, takeaways, and recommendations.



## Project Overview:

### What? 
Discover key drivers of churn, make a predictive model of how likely someone is to churn.

### Why? 
Make data-driven business changes to target our highest areas of opportunity to create the most financial gain/impact.

### How? 
Acquire data, clean it, explore it, model it.


# Executive Summary - Findings & Next Steps

Key Drivers of Churn: Fiber, Electronic Check, Monthly Charges

#### Recommendations: 
- Text reminders of upcoming bills
- small rebate for autopay
- normalization of monthly charges

# Acquire Data

In [2]:
# Queues the sql database for telco data and writes to csv.
# Only reads the csv if it exists
telco_df = acquire.get_telco_data()

# define target variable
target_var = 'churn_Yes'

# show info/standard statistics for the dataframe
print(telco_df.info())
print(telco_df.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7043 non-null   int64  
 1   contract_type_id          7043 non-null   int64  
 2   payment_type_id           7043 non-null   int64  
 3   internet_service_type_id  7043 non-null   int64  
 4   customer_id               7043 non-null   object 
 5   gender                    7043 non-null   object 
 6   senior_citizen            7043 non-null   int64  
 7   partner                   7043 non-null   object 
 8   dependents                7043 non-null   object 
 9   tenure                    7043 non-null   int64  
 10  phone_service             7043 non-null   object 
 11  multiple_lines            7043 non-null   object 
 12  online_security           7043 non-null   object 
 13  online_backup             7043 non-null   object 
 14  device_p

Acquire keypoints:
   - Wrote SQL to queue the Codeup database, write the results to csv. If csv already
      exists, read from csv instead.
    - The initial dataframe has 25 columns and 7043 rows.
    - Both monthly and total charges are the same unit of measure, although tenure is not.
    - Total charges also has a much higher range than both monthly_charges and tenure.           Scaling might be worth exploring in the future.
    - Total charges has a datatype of 'object' when it should be float. Fix in prepare.

# Prepare

In [3]:
# returns clean dataframe, quantitative/categorical columns for 
# future function use. 
telco_df, categories, quant_cols, u_df = prepare.prep_telco(telco_df)
telco_df.shape

(7032, 24)

In [4]:
# split data into training, validate and test splits
train, validate, test = split.train_validate_test_split(u_df, 'churn')
train_en, validate_en, test_en = prepare.encode_train_validate_test(u_df, train.drop(columns='customer_id'), 
                                                                    validate.drop(columns='customer_id'),
                                                                    test.drop(columns='customer_id'))

Preparation Takeaways:
   - There were some 0 values in total_charges that did not make sense to keep, so they were dropped (7043 -> 7032 rows).
   - Converted total_charges to 'float64' so it would be possible to do math and to model with it.
   - Created train, validate and test splits for in-sample testing/exploration, out-of-sample testing, and final model testing.
   - Encoded each train, validate and test.

# Exploration

### visualizations and takeaways 

### statistical testing, hypothesis, etc

# Model and Evaluate

In [5]:
# set baseline
md.Results.baseline = (train_en.churn_Yes==0).mean()
print(f'Baseline accuracy (Churn = 0): {round(md.Results.baseline, 4) * 100}%')

Baseline accuracy (Churn = 0): 73.42999999999999%


If I predicted that someone would not churn, I'd have an accuracy of 73.43%
That is the baseline accuracy that I am trying to outperform.

I'm going to try Logistic Regression, Random Forests, Decision Trees, and KNearest Neighbors to try and create a predicitve classification model.

I've written functions to adjust the parameters for the following:
   - Decision Trees: 
        - depth
   - Random Forests:
        - depth
        - min samples leaf
   - KNearestNeighbor:
        - n_neighbors
        - weights (uniform, distance)
   - Logistic Regression:
        - C
        - solver (lbgfs, liblinear)
        - fit_intercept, intercept_scaling
        
Random State set to 123 where possible.

In [6]:
# create iterations of models, group them into Result objects that I made in modeling.py
dt_mods, rf_mods, knn_mods, lr_mods = md.all_reports(train_en, validate_en, test_en, target_var)

In [7]:
# get highest performing model types while specifying a limiting % diff between train/validate
md.Results.total_summary[md.Results.total_summary.percent_diff < 3.5].groupby('model_type').max()

,depth,train_accuracy,validate_accuracy,difference,percent_diff,min_samples_leaf,n_nearest_neighbor,C
model_type,,,,,,,,
decision_tree,4.0,0.800610,0.777844,0.022766,2.84,NaN,NaN,NaN
knn_uniform,NaN,0.801372,0.775474,0.025898,3.23,NaN,14.0,NaN
logistic_regression,NaN,0.804420,0.796209,0.008211,1.02,NaN,NaN,10.0
random_forests,8.0,0.822200,0.795616,0.027174,3.33,15.0,NaN,NaN


Takeaways from modeling:
- all models seemed to beat baseline
- knn_distance had so much variance that it wouldn't show up under the limiting condition of 3.5% difference between train and validate.
- My logistic regression model performed best with a validate accuracy of 79.62%. This model beats baseline accuracy by 8.43% and is the model I'll use on the test set.

# Test best Model

In [8]:
logit = LogisticRegression(C=10, random_state=123, fit_intercept=False, intercept_scaling=7.5)
x_train = train_en.drop(columns=[target_var, 'customer_id'])
y_train = train_en[target_var]

logit = logit.fit(x_train, y_train)

X_test = test_en.drop(columns=[target_var, 'customer_id'])
Y_test = test_en[target_var]

y_pred_test = logit.predict(X_test)

accuracy = logit.score(X_test, Y_test)

Test Result Takeaways:
   - Accuracy of 80.67% against test set, beating baseline by 9.86%.

# Create Predictions csv

In [9]:
proba_test = logit.predict_proba(X_test)
churn_proba = proba_test[:,1]

predictions = pd.DataFrame()
predictions['customer_id'] = test_en.customer_id
predictions['prediction'] = y_pred_test
predictions['probability'] = churn_proba

predictions.to_csv('predictions.csv')

# Conclusions & Takeaways